In [1]:
#importing libraries
import re
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models import Word2Vec
from scipy  import spatial
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mithi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mithi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:

f=open('text.txt','r')

In [4]:
#breaking the text into sentances
text=[]
for line in f:
    text.append(line)

In [5]:
def split(text):
    sentances=[]
    for line in text:
        sentances=line.split('.')
    return sentances    

In [6]:
def clean(sentances):
    corpus=[]
    for item in sentances:
        
        sentance=re.sub('[^a-zA-Z]',' ',item)
        sentance=sentance.split()
        
        stop_words=stopwords.words('english')
        s=[ wnl.lemmatize(word) for word in sentance if word not in set(stop_words)]
        review=' '.join(s)
        corpus.append(review)
    return corpus
    

In [7]:
def vectorize(sentances):
    sentance_tokens=[[word for word in sentance.split(' ')] for sentance in sentances]
    model1 = Word2Vec(sentance_tokens, min_count = 1, 
                              vector_size = 1, window = 5)
    tokens=[[model1.wv[word][0] for word in words]for words in sentance_tokens]
    max_len=max([len(words)for words in tokens])

    vectors=[np.pad(words,(0,max_len-len(words)),'constant')for words in tokens]
    
    return vectors
    

In [8]:
def similarity(vectors):
    similarity_matrix=np.zeros([len(vectors),len(vectors)])
    #calculating the spatial distance 
    for i,row_embedding in enumerate(vectors):
        for j,column_embedding in enumerate(vectors):
             similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
    return similarity_matrix;            
    

In [11]:
def summarizer(text,lines):
    
    sentances=split(text)
    
    sentance=clean(sentances)
    
    vectors=vectorize(sentance)
    
    matrix=similarity(vectors)
    
    TextRank=np.ones([len(matrix[0])])
    TextRank.reshape(13,)
    
    for i in range(len(matrix[0])):
        for j in range(len(matrix[0])):
            if (j!=i):
                relativity=0.85*(TextRank[j]*matrix[i][j])
        TextRank[i]=(1-0.85)+relativity     
        
    top_sentance={sentance:TextRank[index] for index ,sentance in enumerate(sentences)}
    top=dict(sorted(top_sentance.items(), key=lambda x: x[1], reverse=True)[:lines])
    
    summary=[]
    for sent in sentances:
        if sent in top.keys():
            summary.append(sent)
            
     
    return summary
            
    
    
    
    
    
    
    
    
    
    

In [13]:
summarizer(text,6)

['In its most general sense, the term "world" refers to the totality of entities, to the whole of reality or to everything that is',
 ' In scientific cosmology the world or universe is commonly defined as "[t]he totality of all space and time; all that is, has been, and will be"',
 ' Phenomenology, starting from the horizon of co-given objects present in the periphery of every experience, defines the world as the biggest horizon or the "horizon of all horizons"',
 " Theology conceptualizes the world in relation to God, for example, as God's creation, as identical to God or as the two being interdependent",
 ' Cosmogony is the field that studies the origin or creation of the world while eschatology refers to the science or doctrine of the last things or of the end of the world',
 '']